In [1]:
from collections import defaultdict
import json
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import scipy
from sklearn.metrics import average_precision_score
import sys

sys.path.insert(0, '../Common/')
from COCOWrapper import COCOWrapper
from Dataset import ImageDataset, my_dataloader
from ModelWrapper import ModelWrapper
from ResNet import get_model


In [2]:
data_dir = '/home/gregory/Datasets/unrel'

modes = {'initial-tune': 'Baseline', 'spire': 'SPIRE', 'fs-3': 'FS'}
trials = [0,1,2,3,4,5,6,7]

anns = scipy.io.loadmat('{}/annotations.mat'.format(data_dir))['annotations']

coco = COCOWrapper(mode = 'val')

In [3]:
# Get the image labels and locations for UnRel
files = []
label_dict = defaultdict(list)

for ann in anns:
    # len(o) = 4
    # o[0] -> filename
    # o[2][0][0][0][0] -> info about objects, their locations, and relationship
    o = ann[0][0][0]
    y = o[2][0][0][0][0]
    
    file = '{}/images/{}'.format(data_dir, o[0][0])
    files.append(file)
    
    label_dict[y[0][0]].append(file)
    label_dict[y[1][0]].append(file)


In [4]:
# Find the common objects that SPIRE identified as 'main' for some SP
with open('./2-Models/HPS/spire/spire.json', 'r') as f:
    mains = json.load(f)

tmp = []
for main in mains:
    tmp.append(main.replace('+', ' '))
    
mains = tmp
    
mains = np.intersect1d(mains, list(label_dict))
print(mains)

['bench' 'dog' 'skateboard' 'tie' 'truck']


In [5]:
# Setup the labels
y = []
for file in files:
    y_tmp = np.zeros((91))
    for main in mains:
        index = coco.get_class_id(main)
        if file in label_dict[main]:
            y_tmp[index] = 1
    y.append(y_tmp)

# Get the model's predictions
out = {}
for mode in modes:
    out_mode = defaultdict(list)
    for trial in trials:  
        model_dir = './2-Models/Models/{}/trial{}/model.pt'.format(mode, trial)

        model, _ = get_model(mode = 'tune', parent = model_dir, out_features = 91)
        model.eval()
        model.cuda()

        wrapper = ModelWrapper(model)

        dataset = ImageDataset(files, y)
        dataloader = my_dataloader(dataset)
        y_hat, y_true = wrapper.predict_dataset(dataloader)
        
        for main in mains:
            index = coco.get_class_id(main)
            v = average_precision_score(y_true[:, index], y_hat[:, index])
            out_mode[main].append(v)
    out[mode] = out_mode    

# Get results
print('Results per object')
for main in mains:
    print()
    print('Object: ', main)
    for mode in modes:
        v = out[mode][main]
        print(modes[mode], np.round(np.mean(v), 3), np.round(np.std(v), 4))
        
print()
print()
print('Aggregated results')
print()

for mode in modes:
    tmp = []
    for main in mains:
        tmp.append(np.mean(out[mode][main]))
    print(modes[mode], np.round(np.mean(tmp), 3))

Results per object

Object:  bench
Baseline 0.457 0.0655
SPIRE 0.455 0.0472
FS 0.452 0.0206

Object:  dog
Baseline 0.86 0.0161
SPIRE 0.852 0.0095
FS 0.84 0.0166

Object:  skateboard
Baseline 0.098 0.0534
SPIRE 0.152 0.0433
FS 0.139 0.0499

Object:  tie
Baseline 0.16 0.0573
SPIRE 0.213 0.0662
FS 0.188 0.0623

Object:  truck
Baseline 0.37 0.0585
SPIRE 0.391 0.0531
FS 0.363 0.0341


Aggregated results

Baseline 0.389
SPIRE 0.413
FS 0.396
